<a href="https://colab.research.google.com/github/fender8185/Stock-predict/blob/main/lr3stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install FinMind==1.5.4

In [31]:
import tensorflow as tf
import numpy as np
import FinMind
from FinMind.data import DataLoader
from FinMind import plotting
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from datetime import date


In [ ]:
params = {
        'stock_id': 1234,
        'start_date': '2020-01-01',
        'end_date': date.today()

    }

data_loader = DataLoader()
df_holding = data_loader.taiwan_stock_holding_shares_per(**params)
df_holding = df_holding.pivot(index='date', columns='HoldingSharesLevel', values='percent').reset_index()
stock_data = data_loader.taiwan_stock_daily(**params)
df = df_holding.merge(stock_data, on='date', how='outer')
selected_columns = ['date', '1,000-5,000', '1-999', '10,001-15,000', '100,001-200,000',
       '15,001-20,000', '20,001-30,000', '200,001-400,000', '30,001-40,000',
       '40,001-50,000', '400,001-600,000', '5,001-10,000', '50,001-100,000',
       '600,001-800,000', '800,001-1,000,000', 'more than 1,000,001', 'close']

# Remove rows where all shareholder ratios are NaN or 0
df = df[selected_columns]

df[[ 'close']] = df[[ 'close']].replace(0, np.nan).fillna(method='ffill')

valid_ratio_indices = df[df.columns.difference(['date', 'close'])].dropna(how='all').index
df = df.loc[valid_ratio_indices]
all_dates = df['date'].values[1:]
df.drop(columns=['date'], inplace=True)
print(df)
numpy_array = df.values


In [59]:
# 計算價差
price_diff = numpy_array[1:, -1] - numpy_array[:-1, -1]
# 更新 y
y = price_diff
# X 仍然是前一天的特徵值，但需要去掉最後一天的數據，以確保與 y 的長度相同
X = numpy_array[:-2,:-1]

split_index = int(len(X) * 0.8)  # 80% 資料作為訓練集
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = all_dates[:split_index]
test_dates = all_dates[split_index+1:]  # 因為 y 是從第二天開始的，所以 test_dates 也需要從第二天開始

#normalization 效果竟然比standard效果好？
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# 縮放目標值
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()
'''
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()
'''
# 定義模型參數
W = tf.Variable(tf.random.normal([X_train_scaled.shape[1], 1], dtype=tf.float64), dtype=tf.float64, name="weight")
#b = tf.Variable(tf.random.normal([1], dtype=tf.float64), dtype=tf.float64, name="bias")
b = tf.Variable(tf.zeros([1], dtype=tf.float64), dtype=tf.float64, name="bias")

# 定義模型
def linear_regression(X):
    return tf.matmul(X, W) + b

# 定義損失
def mean_square(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - tf.cast(y_true, tf.float64)))
def mean_absolute_error(y_pred, y_true):
    return tf.reduce_mean(tf.abs(y_pred - tf.cast(y_true, tf.float64)))

# 定義優化器
#optimizer = tf.optimizers.SGD(learning_rate=0.1)
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# 訓練模型
n_epochs = 15000
for epoch in range(n_epochs):
    with tf.GradientTape() as tape:
        y_pred = linear_regression(X_train_scaled)
        loss = mean_absolute_error(y_pred, y_train_scaled)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
    #print(W)
# 評估模型
y_pred_test_scaled = linear_regression(X_test_scaled)
#print(X_test_scaled)
#print(y_pred_test_scaled)
#mse_test = mean_absolute_error(y_pred_test_scaled, y_test_scaled)
#print(f"Mean Squared Error on Scaled Test Data: {mse_test.numpy()}")
#print(y_pred)
# 反縮放預測值回原始尺度
y_pred_test_original = scaler_y.inverse_transform(y_pred_test_scaled.numpy())
y_pred_train_original = scaler_y.inverse_transform(y_pred.numpy())
#mse_test_original = mean_squared_error(y_test, y_pred_test_original)
#print(f"Mean Squared Error on Original Test Data: {mse_test_original}")

#覺得怪怪的但還是做了
# 使用模型對縮放後的訓練數據進行預測
#y_pred_train_scaled = linear_regression(X_train_scaled)
#y_pred_train_original = scaler_y.inverse_transform(y_pred_train_scaled.numpy())

#print(y_pred_train_original)

In [60]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

# 添加訓練數據到第一個子圖
fig.add_trace(go.Scatter(x=train_dates, y=y_train, mode='lines', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=train_dates, y=y_pred_train_original.flatten(), mode='lines', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

# 添加測試數據的真實值到第二個子圖
fig.add_trace(go.Scatter(x=test_dates, y=y_test, mode='lines', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=test_dates, y=y_pred_test_original.flatten(), mode='lines', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)

fig.update_layout(hovermode='x unified')

# 顯示圖形
fig.show()


In [64]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 使用原始的股價計算累積的股價
cumulative_y_train_original = numpy_array[:split_index, -1]
cumulative_y_test_original = numpy_array[split_index+1:-1, -1]

# 昨天的實際close＋predict
cumulative_y_pred_train = cumulative_y_train_original[:] + y_pred_train_original[:].flatten()
cumulative_y_pred_test = cumulative_y_test_original[:] + y_pred_test_original[:].flatten()

fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

# 添加訓練數據到第一個子圖
fig.add_trace(go.Scatter(x=train_dates[1:], y=cumulative_y_train_original[1:], mode='lines', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=train_dates[1:], y=cumulative_y_pred_train, mode='lines', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

# 添加測試數據的真實值到第二個子圖
fig.add_trace(go.Scatter(x=test_dates[:-1], y=cumulative_y_test_original[1:], mode='lines', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=test_dates[:-1], y=cumulative_y_pred_test, mode='lines', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)


# 設置懸停模式
fig.update_layout(hovermode='x unified')
# 顯示圖形
fig.show()
